In [13]:
#!/usr/bin/env python3
# For Jupyter Notebooks / IPython

import asyncio
from datetime import datetime, timezone
from ib_insync import IB, Stock, util
import pandas as pd

# -------- Config --------
IB_HOST = '127.0.0.1'
IB_PORT = 7496       # 7497 = paper, 7496 = live
CLIENT_ID = 102      # Different from your streamer

# For Jupyter notebooks, use util.startLoop()
util.startLoop()

async def get_historical_data():
    ib = IB()
    await ib.connectAsync(IB_HOST, IB_PORT, clientId=CLIENT_ID)
    print("Connected to IBKR")
    
    # Qualify QQQ
    [qqq] = await ib.qualifyContractsAsync(Stock('QQQ', 'ARCA', 'USD'))
    
    # Get 1-minute bars for last 2 days
    bars_1m = await ib.reqHistoricalDataAsync(
        qqq,
        endDateTime='',  # Empty string = now
        durationStr='2 D',  # 2 days of data
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=False,  # Include pre/post market
        formatDate=1
    )
    
    # Get 5-minute bars for last week
    bars_5m = await ib.reqHistoricalDataAsync(
        qqq,
        endDateTime='',
        durationStr='1 W',  # 1 week of data
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1
    )
    
    print(f"\n1-MINUTE BARS (last 10):")
    print("─" * 60)
    for bar in bars_1m[-10:]:  # Last 10 bars
        print(f"{bar.date} | O:{bar.open:7.2f} H:{bar.high:7.2f} "
              f"L:{bar.low:7.2f} C:{bar.close:7.2f} V:{bar.volume:8.0f}")
    
    print(f"\n5-MINUTE BARS (last 10):")
    print("─" * 60)
    for bar in bars_5m[-10:]:
        print(f"{bar.date} | O:{bar.open:7.2f} H:{bar.high:7.2f} "
              f"L:{bar.low:7.2f} C:{bar.close:7.2f} V:{bar.volume:8.0f}")
    
    # Convert to pandas DataFrame
    df_1m = pd.DataFrame([{
        'date': bar.date,
        'open': bar.open,
        'high': bar.high,
        'low': bar.low,
        'close': bar.close,
        'volume': bar.volume
    } for bar in bars_1m])
    
    df_5m = pd.DataFrame([{
        'date': bar.date,
        'open': bar.open,
        'high': bar.high,
        'low': bar.low,
        'close': bar.close,
        'volume': bar.volume
    } for bar in bars_5m])
    
    # Save to CSV if needed
    df_1m.to_csv('qqq_1m.csv', index=False)
    df_5m.to_csv('qqq_5m.csv', index=False)
    print(f"\nSaved {len(df_1m)} 1m bars to qqq_1m.csv")
    print(f"Saved {len(df_5m)} 5m bars to qqq_5m.csv")
    
    ib.disconnect()
    return df_1m, df_5m

# SYNCHRONOUS VERSION (easier for notebooks)
def get_historical_sync():
    ib = IB()
    ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
    print("Connected to IBKR")
    
    # Qualify QQQ
    qqq = Stock('QQQ', 'ARCA', 'USD')
    ib.qualifyContracts(qqq)
    
    # Get 1-minute bars
    bars_1m = ib.reqHistoricalData(
        qqq,
        endDateTime='',
        durationStr='2 D',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1  # Add this to get datetime objects
    )
    
    # Get 5-minute bars
    bars_5m = ib.reqHistoricalData(
        qqq,
        endDateTime='',
        durationStr='1 W',
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1  # Add this to get datetime objects
    )
    
    # Convert to DataFrame - handle timezone issues
    df_1m = pd.DataFrame([{
        'date': pd.Timestamp(bar.date).tz_localize(None),  # Remove timezone
        'open': bar.open,
        'high': bar.high,
        'low': bar.low,
        'close': bar.close,
        'volume': bar.volume
    } for bar in bars_1m])
    
    df_5m = pd.DataFrame([{
        'date': pd.Timestamp(bar.date).tz_localize(None),  # Remove timezone
        'open': bar.open,
        'high': bar.high,
        'low': bar.low,
        'close': bar.close,
        'volume': bar.volume
    } for bar in bars_5m])
    
    print(f"Got {len(df_1m)} 1-minute bars")
    print(f"Got {len(df_5m)} 5-minute bars")
    
    ib.disconnect()
    return df_1m, df_5m

# FOR JUPYTER - Use await directly or the sync version:
# Option 1: Await the async function
# df_1m, df_5m = await get_historical_data()

# Option 2: Use the sync version (easier!)
# df_1m, df_5m = get_historical_sync()

# Quick function to just get recent data - FIXED VERSION
def get_quick_data(symbol='QQQ', days=1, bar_size='1 min'):
    ib = IB()
    ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
    
    stock = Stock(symbol, 'ARCA', 'USD')
    ib.qualifyContracts(stock)
    
    bars = ib.reqHistoricalData(
        stock,
        endDateTime='',
        durationStr=f'{days} D',
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=False,
        formatDate=2  # Use string format to avoid timezone issues
    )
    
    # Create DataFrame with strings first, then convert
    data = []
    for bar in bars:
        data.append({
            'date': str(bar.date),  # Keep as string
            'open': float(bar.open),
            'high': float(bar.high),
            'low': float(bar.low),
            'close': float(bar.close),
            'volume': int(bar.volume),
            'average': float(bar.average),
            'barCount': int(bar.barCount)
        })
    
    df = pd.DataFrame(data)
    
    # Optionally convert date to datetime without timezone
    # df['date'] = pd.to_datetime(df['date'], format='%Y%m%d %H:%M:%S')
    
    ib.disconnect()
    
    return df

# Even simpler version - no pandas datetime at all
def get_data_simple(symbol='QQQ', days=1, bar_size='1 min'):
    ib = IB()
    ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
    
    stock = Stock(symbol, 'ARCA', 'USD')
    ib.qualifyContracts(stock)
    
    bars = ib.reqHistoricalData(
        stock,
        endDateTime='',
        durationStr=f'{days} D',
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=False
    )
    
    # Just return the raw bars - no DataFrame conversion
    ib.disconnect()
    
    # Access data like: bars[0].open, bars[0].close, etc.
    return bars

# Example usage in notebook cells:
# Cell 1:
# df = get_quick_data('QQQ', days=2, bar_size='5 mins')
# df.tail(20)

# Cell 2: Plot the data
# import matplotlib.pyplot as plt
# df['close'].plot(figsize=(12,6))
# plt.title('QQQ 5-minute bars')
# plt.show()

In [18]:
# 🎯 ENHANCED DATA LOADING SYSTEM - No more timeouts!
# Uses smart fallback: Live IBKR → CSV Backfill → Error handling

# Import the enhanced data loader
import sys
sys.path.append('/Users/mmirandi/Desktop/Finance')
from qqq_data_loader import get_qqq_data, quick_csv_load, quick_live_test

print("🔄 SMART QQQ DATA LOADING")
print("=" * 50)

# Option 1: Smart loading (tries live first, falls back to CSV)
# df_1m, df_5m, df_4h = get_qqq_data(prefer_live=True, live_days=14)

# Option 2: CSV only (fastest, most reliable)
df_1m, df_5m, df_4h = quick_csv_load()

# Option 3: Force live data attempt
# df_1m, df_5m, df_4h = get_qqq_data(prefer_live=True, live_days=2)  # Smaller chunks

print(f"\n✅ DATA LOADING COMPLETE")
print(f"   Ready for ORB analysis with 4H confluence!")

# Quick data validation
for name, df in [('1m', df_1m), ('5m', df_5m), ('4h', df_4h)]:
    if not df.empty:
        print(f"   {name}: {len(df):,} bars ✅")
    else:
        print(f"   {name}: No data ❌")
        
# Set up data for ORB system
if not df_4h.empty:
    print(f"\n🎯 4H Confluence Data Check:")
    print(f"   4H bars available: {len(df_4h)}")
    print(f"   Sufficient for RSI (14): {'✅' if len(df_4h) >= 20 else '❌'}")
    print(f"   Sufficient for MACD (26): {'✅' if len(df_4h) >= 35 else '❌'}")

🔄 SMART QQQ DATA LOADING
📂 Loading CSV data only...
📂 Loading 1m data from qqq_1m_backfill.csv
   ✅ Loaded 66,059 bars (2025-05-16 08:00:00 to 2025-08-25 23:59:00)
📂 Loading 5m data from qqq_5m_backfill.csv
   ✅ Loaded 13,596 bars (2025-05-14 08:00:00 to 2025-08-25 23:55:00)
📂 Loading 4h data from qqq_4h_backfill.csv
   ✅ Loaded 348 bars (2025-04-22 08:00:00 to 2025-08-25 20:00:00)

✅ DATA LOADING COMPLETE
   Ready for ORB analysis with 4H confluence!
   1m: 66,059 bars ✅
   5m: 13,596 bars ✅
   4h: 348 bars ✅

🎯 4H Confluence Data Check:
   4H bars available: 348
   Sufficient for RSI (14): ✅
   Sufficient for MACD (26): ✅


In [15]:
df_5m

""


In [16]:
df_1m.dtypes

Series([], dtype: object)

In [17]:
# 🧪 VALIDATION: Test 4H data quality and prepare for ORB system
import pandas as pd
import numpy as np

print("🔍 VALIDATING 4H DATA FOR ORB CONFLUENCE")
print("=" * 50)

# Convert date strings to datetime for all dataframes
for df_name, df in [('1m', df_1m), ('5m', df_5m), ('4h', df_4h)]:
    if df['date'].dtype == 'object':
        df['date'] = pd.to_datetime(df['date'])
        print(f"✅ Converted {df_name} dates to datetime")

# Check 4H data has sufficient history for indicators
print(f"\n📈 4H Data Analysis:")
print(f"   Total 4H bars: {len(df_4h)}")
print(f"   Date range: {df_4h['date'].min()} to {df_4h['date'].max()}")
print(f"   Days covered: {(df_4h['date'].max() - df_4h['date'].min()).days}")

# Quick indicator viability check
print(f"   RSI viability: {'✅ YES' if len(df_4h) >= 20 else '❌ NO'} (need 20+ bars, have {len(df_4h)})")
print(f"   MACD viability: {'✅ YES' if len(df_4h) >= 35 else '❌ NO'} (need 35+ bars, have {len(df_4h)})")

# Preview 4H OHLCV for manual inspection  
print(f"\n🔍 Recent 4H Bars:")
recent_4h = df_4h.tail(10)[['date', 'open', 'high', 'low', 'close', 'volume']]
print(recent_4h.to_string(index=False))

# Check for any data issues
print(f"\n🚨 Data Quality Issues:")
null_check = df_4h.isnull().sum().sum()
print(f"   Null values: {null_check} {'❌ ISSUE' if null_check > 0 else '✅ CLEAN'}")

zero_volume = (df_4h['volume'] == 0).sum()
print(f"   Zero volume bars: {zero_volume} {'❌ ISSUE' if zero_volume > len(df_4h)*0.1 else '✅ ACCEPTABLE'}")

# Test basic calculation compatibility
print(f"\n🧮 Basic Calculations Test:")
try:
    df_test = df_4h.copy()
    df_test['sma_5'] = df_test['close'].rolling(5).mean()
    df_test['price_change'] = df_test['close'].pct_change()
    print("   ✅ Calculations working correctly")
    print(f"   Recent SMA5: {df_test['sma_5'].iloc[-1]:.2f}")
    print(f"   Recent change: {df_test['price_change'].iloc[-1]*100:.2f}%")
except Exception as e:
    print(f"   ❌ Calculation error: {e}")

print(f"\n🎯 READY FOR ORB SYSTEM INTEGRATION")
print(f"   All timeframes available: 1m ✅, 5m ✅, 4h ✅")
print(f"   Data quality: {'✅ PASSED' if null_check == 0 else '⚠️ REVIEW NEEDED'}")

🔍 VALIDATING 4H DATA FOR ORB CONFLUENCE


KeyError: 'date'